In [49]:
from swissisoform.genome import GenomeHandler
from swissisoform.visualize import GenomeVisualizer
from swissisoform.alternative_isoforms import AlternativeIsoform
from swissisoform.mutations import MutationHandler, analyze_mutations
import pandas as pd
import os

In [50]:
genome = GenomeHandler(
    "../data/genome_data/hg38.fa", "../data/genome_data/hg38.ncbiRefSeq.gtf"
)

alt_isoforms = AlternativeIsoform()
alt_isoforms.load_bed(
    "../data/ribosome_profiling/full_truncations_JL_cleaned.bed"
)

mutation_handler = MutationHandler()

KeyboardInterrupt: 

In [ ]:
gene_name = "NTHL1"
print(f"Processing gene: {gene_name}")

os.makedirs(f"./{gene_name}", exist_ok=True)

# Get alternative isoform features
print("Getting alternative features...")
alt_features = alt_isoforms.get_visualization_features(gene_name)

if alt_features.empty:
    print("No alternative features found")
else:
    print(f"Found {len(alt_features)} alternative features")
    print("\nAlternative Features:")
    print(alt_features)

Processing gene: NTHL1
Getting alternative features...
Found 2 alternative features

Alternative Features:
    chromosome      source       feature_type    start      end  score strand  \
704      chr16  truncation  alternative_start  2047709  2047845      0      -   
705      chr16  truncation  alternative_start  2046202  2046366      0      -   

    frame            gene_id          transcript_id gene_name start_codon  
704     .  ENSG00000065057.7  ENSG00000065057.7_alt     NTHL1         AUG  
705     .  ENSG00000065057.7  ENSG00000065057.7_alt     NTHL1         AUG  


In [ ]:
print("Getting transcript information...")
transcript_info = genome.get_transcript_ids(gene_name)

if transcript_info.empty:
    print("No transcript info found")
else:
    print(f"Found {len(transcript_info)} transcripts")
    print("\nTranscript Information:")
    print(transcript_info)

Getting transcript information...
Found 4 transcripts

Transcript Information:
          transcript_id chromosome    start      end strand
1360102  XM_047434171.1      chr16  2039820  2047834      -
1360119     NM_002528.7      chr16  2039820  2047834      -
1360136  NM_001318193.2      chr16  2039820  2047834      -
1360151  NM_001318194.2      chr16  2039820  2047834      -


In [ ]:
print("Fetching mutation data...")
all_mutations = await mutation_handler.get_visualization_ready_mutations(
    gene_name=gene_name,
    alt_features=None,  # Don't filter by alt_features yet
    sources=["clinvar"],
    aggregator_csv_path=None,
)

if all_mutations is None or all_mutations.empty:
    print("No mutations found for this gene")
    all_mutations = pd.DataFrame()
else:
    print(f"Found {len(all_mutations)} total mutations for this gene")


Fetching mutation data...


/lab/barcheese01/mdiberna/swissisoform/src/swissisoform/mutations.py:458: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")


Found 392 total mutations for this gene


In [ ]:
# Define impact types for filtering (same as analyze_truncations.py)
impact_types = {
    "clinvar": ["missense variant", "nonsense variant", "frameshift variant"]
}

# Apply impact type filtering if specified
if impact_types:
    print(f"Filtering for impact types: {impact_types}")
    filtered_mutations = pd.DataFrame()
    
    for source, impacts in impact_types.items():
        source_mutations = all_mutations[
            all_mutations["source"].str.lower() == source.lower()
        ]

        if not source_mutations.empty:
            filtered_source = source_mutations[
                source_mutations["impact"].isin(impacts)
            ]
            filtered_mutations = pd.concat([filtered_mutations, filtered_source])
            
    if not filtered_mutations.empty:
        print(f"After filtering: {len(filtered_mutations)} mutations")
        all_mutations_filtered = filtered_mutations
    else:
        print("No mutations match the filter criteria")
        all_mutations_filtered = pd.DataFrame()
else:
    all_mutations_filtered = all_mutations.copy()


Filtering for impact types: {'clinvar': ['missense variant', 'nonsense variant', 'frameshift variant']}
After filtering: 235 mutations


In [ ]:
# Create transcript-truncation pairs based on overlap
print("Creating transcript-truncation pairs...")
transcript_truncation_pairs = []

for _, transcript in transcript_info.iterrows():
    transcript_id = transcript["transcript_id"]
    transcript_start = transcript["start"]
    transcript_end = transcript["end"]
    transcript_chromosome = transcript["chromosome"]
    transcript_strand = transcript["strand"]

    # Check each truncation feature for overlap with this transcript
    for idx, truncation in alt_features.iterrows():
        trunc_start = truncation["start"]
        trunc_end = truncation["end"]
        trunc_chrom = truncation["chromosome"]

        # Skip if chromosomes don't match
        if transcript_chromosome != trunc_chrom:
            continue

        # Check for overlap
        if not (transcript_end < trunc_start or transcript_start > trunc_end):
            # Create an entry for this transcript-truncation pair
            truncation_id = f"trunc_{idx}"

            # If we have more identifiable information about the truncation, use it
            if "start_codon" in truncation and not pd.isna(truncation["start_codon"]):
                truncation_id = f"trunc_{truncation['start_codon']}_{trunc_start}_{trunc_end}"

            transcript_truncation_pairs.append({
                "transcript_id": transcript_id,
                "truncation_id": truncation_id,
                "truncation_idx": idx,
                "transcript_start": transcript_start,
                "transcript_end": transcript_end,
                "transcript_strand": transcript_strand,
                "truncation_start": trunc_start,
                "truncation_end": trunc_end,
            })

if not transcript_truncation_pairs:
    print("No transcripts overlap with truncation regions")
else:
    print(f"Found {len(transcript_truncation_pairs)} transcript-truncation pairs across {len(transcript_info)} transcripts")


Creating transcript-truncation pairs...
Found 8 transcript-truncation pairs across 4 transcripts


In [60]:
pair_results = []
if transcript_truncation_pairs:
    visualizer = GenomeVisualizer(genome)
    
    print("Analyzing mutations for each transcript-truncation pair...")
    
    for pair_idx, pair in enumerate(transcript_truncation_pairs, 1):
        transcript_id = pair["transcript_id"]
        truncation_id = pair["truncation_id"]
        truncation_idx = pair["truncation_idx"]
        
        # Get truncation start and end positions
        trunc_start = pair["truncation_start"]
        trunc_end = pair["truncation_end"]
        
        # Initialize default counts for mutation categories
        mutation_categories = {}
        desired_impact_types = []
        
        # Get the list of desired impact types
        if impact_types:
            for source, impacts in impact_types.items():
                desired_impact_types.extend(impacts)
                
        # Ensure all desired impact types have columns, even if zero
        for impact_type in desired_impact_types:
            category_key = f"mutations_{impact_type.replace(' ', '_').lower()}"
            mutation_categories[category_key] = 0
            
            # Also create empty columns for ClinVar IDs for each impact type
            impact_key = f"clinvar_ids_{impact_type.replace(' ', '_').lower()}"
            mutation_categories[impact_key] = ""
        
        # Create a truncation-specific filter for mutations
        if not all_mutations.empty:
            # Filter mutations to only those in this truncation region
            pair_mutations = all_mutations[
                (all_mutations["position"] >= trunc_start) &
                (all_mutations["position"] <= trunc_end)
            ].copy()
            
            pair_mutation_count = len(pair_mutations)
            
            if pair_mutation_count > 0:
                print(f"  ├─ {transcript_id} × {truncation_id}: {pair_mutation_count} mutations")
                
                clinvar_ids = []
                
                # Count by impact category
                for impact in pair_mutations["impact"].unique():
                    if pd.isna(impact):
                        continue
                    
                    # Get mutations for this impact
                    impact_mutations = pair_mutations[pair_mutations["impact"] == impact]
                    category_count = len(impact_mutations)
                    
                    # Store count for this impact type
                    category_key = f"mutations_{impact.replace(' ', '_').lower()}"
                    mutation_categories[category_key] = category_count
                    
                    # Store ClinVar IDs for this impact type
                    if "variant_id" in impact_mutations.columns:
                        impact_ids = impact_mutations["variant_id"].dropna().unique().tolist()
                        # Convert to strings (to handle numeric IDs), filter empty strings
                        impact_ids = [str(id).strip() for id in impact_ids if str(id).strip()]
                        
                        # Add to the category-specific IDs
                        if impact_ids:
                            impact_key = f"clinvar_ids_{impact.replace(' ', '_').lower()}"
                            mutation_categories[impact_key] = ",".join(impact_ids)
                
                # Collect all ClinVar IDs for this truncation region
                if "variant_id" in pair_mutations.columns:
                    clinvar_ids = pair_mutations["variant_id"].dropna().unique().tolist()
                    clinvar_ids = [str(id).strip() for id in clinvar_ids if str(id).strip()]
                
                # Add results for this pair with detailed mutation categories
                pair_results.append({
                    "transcript_id": transcript_id,
                    "truncation_id": truncation_id,
                    "truncation_start": trunc_start,
                    "truncation_end": trunc_end,
                    "mutation_count_total": pair_mutation_count,
                    "clinvar_variant_ids": ",".join(clinvar_ids) if clinvar_ids else "",
                    **mutation_categories,
                })
            else:
                # No mutations for this pair, still record it with zeros
                pair_results.append({
                    "transcript_id": transcript_id,
                    "truncation_id": truncation_id,
                    "truncation_start": trunc_start,
                    "truncation_end": trunc_end,
                    "mutation_count_total": 0,
                    "clinvar_variant_ids": "",
                    **mutation_categories,  # Include zero counts for all categories
                })
        else:
            # No mutations at all, record with zeros
            pair_results.append({
                "transcript_id": transcript_id,
                "truncation_id": truncation_id,
                "truncation_start": trunc_start,
                "truncation_end": trunc_end,
                "mutation_count_total": 0,
                "clinvar_variant_ids": "",
                **mutation_categories,  # Include zero counts for all categories
            })
            
        # Get the specific truncation for this pair
        if truncation_idx in alt_features.index:
            truncation_feature = alt_features.loc[[truncation_idx]].copy()
        else:
            print(f"  ├─ Warning: Invalid truncation index {truncation_idx}, skipping visualization")
            continue
        
        print(f"  ├─ Visualizing pair {pair_idx}/{len(transcript_truncation_pairs)}: {transcript_id} × {truncation_id}")
        
        # Create directories organized by transcript and truncation
        output_dir = f"./{gene_name}/"
        transcript_dir = f"{output_dir}{transcript_id}"
        os.makedirs(transcript_dir, exist_ok=True)
        
        # Prepare output paths
        pair_base_filename = f"{transcript_id}_{truncation_id}"
        
        # Filter mutations for this specific truncation
        if not all_mutations.empty:
            pair_mutations = all_mutations[
                (all_mutations["position"] >= trunc_start) &
                (all_mutations["position"] <= trunc_end)
            ].copy()
            
            # Create the visualization using only this truncation feature
            print(f"  │  ├─ Creating view with {len(pair_mutations)} mutations")
            visualizer.visualize_transcript(
                gene_name=gene_name,
                transcript_id=transcript_id,
                alt_features=truncation_feature,
                mutations_df=pair_mutations,
                output_file=f"{transcript_dir}/{pair_base_filename}_filtered.png",
            )
            
            print(f"  │  └─ Creating zoomed view")
            visualizer.visualize_transcript_zoomed(
                gene_name=gene_name,
                transcript_id=transcript_id,
                alt_features=truncation_feature,
                mutations_df=pair_mutations,
                output_file=f"{transcript_dir}/{pair_base_filename}_filtered_zoom.png",
                padding=100,
            )
    
    # Save pair results to CSV
    if pair_results:
        results_df = pd.DataFrame(pair_results)
        
        # Organize columns in preferred order
        column_order = [
            "transcript_id",          # Transcript ID first
            "truncation_id",          # Alternative feature ID second
            "truncation_start", 
            "truncation_end",
            "mutation_count_total"    # Total mutation count
        ]
        
        # Add mutation category columns to the order (they start with "mutations_")
        mutation_category_columns = [col for col in results_df.columns if col.startswith("mutations_")]
        column_order.extend(mutation_category_columns)
        
        # Add the ClinVar IDs column 
        clinvar_columns = [col for col in results_df.columns if col.startswith("clinvar")]
        column_order.extend(clinvar_columns)
        
        # Reorder columns that exist in the dataframe
        available_columns = [col for col in column_order if col in results_df.columns]
        
        # Add any remaining columns that weren't explicitly ordered
        remaining_columns = [col for col in results_df.columns if col not in available_columns]
        final_column_order = available_columns + remaining_columns
        
        # Apply the column order and save
        if final_column_order:  # Only reorder if we have columns to reorder
            results_df = results_df[final_column_order]
            
        # Calculate total mutations across all pairs
        total_mutations = sum(pair["mutation_count_total"] for pair in pair_results)
        print(f"  └─ Total mutations across all pairs: {total_mutations}")
            
        results_df.to_csv(f"./{gene_name}/pair_results.csv", index=False)
        print(f"Saved pair results to ./{gene_name}/pair_results.csv")
else:
    print("No transcript-truncation pairs to visualize")

Analyzing mutations for each transcript-truncation pair...
  ├─ XM_047434171.1 × trunc_AUG_2047709_2047845: 56 mutations
  ├─ Visualizing pair 1/8: XM_047434171.1 × trunc_AUG_2047709_2047845
  │  ├─ Creating view with 56 mutations
Features before processing         chromosome                 source feature_type    start      end  \
1360102      chr16  ncbiRefSeq.2022-10-28   transcript  2039820  2047834   
1360103      chr16  ncbiRefSeq.2022-10-28         exon  2039820  2040047   
1360104      chr16  ncbiRefSeq.2022-10-28         3UTR  2039820  2039923   
1360105      chr16  ncbiRefSeq.2022-10-28          CDS  2039927  2040047   
1360106      chr16  ncbiRefSeq.2022-10-28         exon  2040133  2040238   
1360107      chr16  ncbiRefSeq.2022-10-28          CDS  2040133  2040238   
1360108      chr16  ncbiRefSeq.2022-10-28         exon  2043567  2043726   
1360109      chr16  ncbiRefSeq.2022-10-28          CDS  2043567  2043726   
1360110      chr16  ncbiRefSeq.2022-10-28         exon  20